In [3]:
# ! conda install pandas -y
# ! conda install pyfastx -y
# ! conda install biopython -y
# !  conda install -c bioconda clustalo -y
# ! conda install -c bioconda seqtk -y
# ! conda install tqdm -y


In [4]:
import pandas as pd
import numpy as np
import subprocess

from pathlib import Path
from pyfastx import Fastq, Fasta
from tqdm import tqdm
import os
import time
from Bio import pairwise2, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import ClustalOmegaCommandline


In [5]:
run_name = "big_allen"
tidehunter_file = f"data/out/{run_name}/tidehunter/FAO43712_pass_75edb72a_1.consensus.tsv"
backbone_3_file = f"data/out/{run_name}/primes/BB22_40_p3.fasta"

backbone_5_file = f"data/out/{run_name}/primes/BB22_40_p5.fasta"
backbone_full_file = f"/home/dami/data/backbones/backbones/BB22.fa"

print(tidehunter_file)
data = Path(tidehunter_file)
assert data.exists()

fastq = '/home/dami/data/raw_data/JAG6252/fastq_pass/FAO43712_pass_75edb72a_1.fastq'
fq = Fastq(fastq)
bb_fa = Fasta(backbone_full_file)

df = pd.read_csv(str(data), sep="\t")

data/out/big_allen/tidehunter/FAO43712_pass_75edb72a_1.consensus.tsv


In [6]:
df['consSeqLength'] = df['consSeq'].apply(lambda x: len(x))
df.head()

,readName,repN,copyNum,readLen,start,end,consLen,aveMatch,fullLen,subPos,consSeq,consSeqLength
0,e564aadf-cc1e-47c5-bf60-544f0e777510,rep0,8561,39,8560,277,31.2,94.2,0,"86,350,625,895,1171,1453,1723,1993,2256,2536,2...",TCCCACTTCTCTCTTGTTTCCTCTCATGTCATCTTATTTCTGCACA...,277
1,caf5b726-fa63-49be-96b9-66b7114e66c5,rep0,17126,27,17021,388,44.0,96.8,0,"42,430,816,1198,1573,1965,2351,2744,3132,3523,...",GCGCGGACGCGGGTGCCGGGCGGGGGTGTGGAATCAACCCACAGCT...,388
2,94554df4-2a4a-4fd5-862b-2cd5579344b0,rep0,8785,29,8785,275,22.8,97.2,2,"139,528,911,1303,1684,2070,2453,2837,3219,3608...",TCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGC...,275
3,bb80c993-5720-4eff-b339-cf1ba9212ce4,rep0,21922,32,21909,386,62.2,94.3,0,"213,549,902,1260,1623,1978,2362,2730,3096,3443...",CCAACTGGCCAAGACCTGCCCTGTGCAGCTGTGGGTTGATTCCACA...,386
4,96c01dee-d356-42ec-b406-070555a9561e,rep0,5190,109,1009,221,4.0,89.9,0,"351,572,794",CTGTAGTGTCCAGAATTAGGAAATAAATAAAAAGTCTGGCTAGGTG...,221


In [7]:
# ! head -n 4 /home/dami/data/raw_data/JAG6252/fastq_pass/FAO43712_pass_75edb72a_1.fastq

In [8]:
# ! cat /home/dami/data/raw_data/JAG6252/fastq_pass/FAO43712_pass_75edb72a_1.fastq | grep -A 1 df7147fa-f049-4f4b-bb92-d2321ba6f4c3

In [9]:
def create_consensus_fq(fq: str, consensus_data: pd.DataFrame, bb: str, include_bb: bool = True) -> Path:
    
    consensus_fastas = []
    fastq = Fastq(fq)

    for i, row in tqdm(consensus_data.iterrows()):
        readname = row.readName
        repeatlength = row.end
        repeat_no = row.repN

        row_filename = f"data/tmp/{run_name}/{readname}-{repeat_no}.fasta"
        row_file = Path(row_filename)
        
        row_file.parent.mkdir(parents= True, exist_ok=True)
        consensus_fastas.append(row_file)
        
        read = fastq[readname]

        locs = row.subPos.split(',')
        locs = [int(x) for x in locs]

        with open(row_file, "a") as output_handle:
            if include_bb:
                seqrecord_bb = get_backbone_seqrecord(bb)
                SeqIO.write(seqrecord_bb, output_handle,"fasta")

            for loc in locs:
                loc_endpoint= loc+repeatlength
                loc_name = f"{readname}-{row.repN}-{loc}:{loc_endpoint}"
                loc_sequence = read.seq[loc : loc_endpoint]
                # loc_qual = read.qual

                record = SeqRecord(
                    Seq(loc_sequence),
                    id= loc_name,
                    name= loc_name,
                    description= loc_name,
                )
                # write record to file as fasta
                SeqIO.write(
                    record,
                    output_handle,
                    "fasta"
                )
    return consensus_fastas

def get_backbone_seqrecord(bb_file, name="BB22"):
    bb = Fasta(bb_file)
    seqrecord_bb = SeqRecord(
                Seq(bb[name].seq),
                id= bb[name].name,
                name= bb[name].name,
                description= bb[name].name,
            )
    return seqrecord_bb


In [10]:
def run_tcoffee(fastas):
    for fasta in tqdm(fastas):
        subprocess.run(f"t_coffee {fasta} --thread 10".split(' '), check=True, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
        # time.sleep(3)
        # subprocess.run(f"mv {fasta.stem}* {fasta.parent}/".split(' '),check=True)


In [11]:
def create_consensus(in_fq, consensus_data, bb: str, include_bb: bool) -> None:
    cons_fq = create_consensus_fq(in_fq, consensus_data, bb, include_bb)
    run_tcoffee(cons_fq)

In [12]:
create_consensus(fastq, df.head(), backbone_full_file,True)

5it [00:00, 139.89it/s]
  0%|          | 0/5 [00:36<?, ?it/s]


CalledProcessError: Command '['t_coffee', 'data/tmp/big_allen/e564aadf-cc1e-47c5-bf60-544f0e777510-rep0.fasta', '--thread', '10']' returned non-zero exit status 247.

In [ ]:
!t_coffee data/tmp/big_allen/e564aadf-cc1e-47c5-bf60-544f0e777510-rep0.fasta


********************************************************************************
Command line arguments: ['/home/dami/miniconda3/envs/cycloseq/bin/t_coffee', 'data/tmp/big_allen/e564aadf-cc1e-47c5-bf60-544f0e777510-rep0.fasta']
Install folder: /home/dami/miniconda3/envs/cycloseq/lib/t_coffee-11.0.8
********************************************************************************


PROGRAM: T-COFFEE Version_11.00.8cbe486 (2014-08-12 22:05:29 - Revision 8cbe486 - Build 477)
-full_log      	S	[0] 
-genepred_score	S	[0] 	nsd
-run_name      	S	[0] 
-mem_mode      	S	[0] 	mem
-extend        	D	[1] 	1 
-extend_mode   	S	[0] 	very_fast_triplet
-max_n_pair    	D	[0] 	10 
-seq_name_for_quadruplet	S	[0] 	all
-compact       	S	[0] 	default
-clean         	S	[0] 	no
-do_self       	FL	[0] 	0
-do_normalise  	D	[0] 	1000 
-template_file 	S	[0] 
-setenv        	S	[0] 	0
-template_mode 	S	[0] 
-flip          	D	[0] 	0 
-remove_template_file	D	[0] 	0 
-profile_template_file	S	[0] 
-in            	S	[0] 